# All Imports

In [28]:
import os
from dotenv import load_dotenv
import requests
import time
import sys

# Globals

In [27]:
DEFAULT_WR = 0.40
# Can use later for getting better avg/accounting for other gamemodes
EXCLUDED_QUEUE_IDS = {
    0,    # Custom games
    830,  # Co-op vs. AI: Intro bots
    840,  # Co-op vs. AI: Beginner bots
    850,  # Co-op vs. AI: Intermediate bots
    450,  # ARAM
    900,  # ARURF
    920,  # Nexus Blitz
    1300  # Nexus Blitz (old)
    # Add more queue IDs to exclude other game modes
}

# Getting Data

In [22]:
# Set up API (NA1 Region Only)
load_dotenv()
api_key = os.getenv("RIOT_API_KEY")

In [50]:
# Function for detecting and handling limit reached (429) and NOT success (200)
def apiCallHandler(request_url):
    #print(request_url)
    response = requests.get(request_url)

    # Count safeguard mech: shouldn't loop twice
    count = 0
    while response.status_code != 200:
        if(count == 1):
            print("WEIRD ERROR: status 429 happend twice in a row?")
            sys.exit("Stopping all execution")
        if(response.status_code == 429):
            # Not success but is 429 API limit error
            count+=1
            print("sleeping 1s")
            time.sleep(1)
        else:
            # Not success and not 429 API limit error
            print(f"Failed to fetch data: {response.status_code}")
            sys.exit("Stopping all execution")
    # (finally) status of 200
    return response.json()
    

In [59]:
# Function for calculating average win rate of a summoner
def avg_wr_summoner(match_history, summoner_puuid):
    if(len(match_history) == 0):
        return DEFAULT_WR
    total_matches = 0
    win_count = 0
    for match in match_history:
        # Future work: use multithreading for long calls like these
        response = apiCallHandler(f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}")
        # Safeguard just in case Riot includes incorrect PUUID
        try:
            participant = next(p for p in response['info']['participants'] if p['puuid'] == summoner_puuid)
            if(participant['win'] == True):
                win_count+=1
            total_matches+=1
        except StopIteration:
            print("ERROR: StopIteration exception occured, Riot data incorrect?")
            sys.exit("Stopping all execution")
        except:
            print("ERROR: Exception occured, shouldn't be here")
            sys.exit("Stopping all execution")
    return round(win_count/total_matches, 2)
        
    

In [60]:
# Gets Data for a given sumoner name and summoner tag
# Data currently equals: Summoner Level, AVG win rate for past 20 matches, 
# def get_something

# tmp, replace with ID of summoner
summoner_name = "kneesyd"#"portal"
summoner_tag = "NA1"#"only2"

# Get PUUID based on name & tag
account_data = apiCallHandler(f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{summoner_name}/{summoner_tag}?api_key={api_key}')
print(f"Summoner ID: {summoner_name}#{summoner_tag}")
print(f"Summoner PUUID: {account_data['puuid']}")
summoner_puuid = account_data['puuid']

# Get summoners' level using their PUUID
summoner_data = apiCallHandler(f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{summoner_puuid}?api_key={api_key}')
summoner_level = summoner_data['summonerLevel']

# Get summoners' match history using their PUUID, filters only for normal draft games
summoner_match_history = apiCallHandler(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{summoner_puuid}/ids?queue=400&type=normal&start=0&count=20&api_key={api_key}')

# Get summoners' win rate using match history
# Future work: Ensure that no custom, bot, tutorial, or arena/limited game mode matches
# Future work: If match count is < 20, THEN use quickplay and aram stats (assign weights to them)
# Future work: add compatibility for predicting specifically gamemodes other than normal draft 
avg_wr = avg_wr_summoner(summoner_match_history, summoner_puuid)
print(avg_wr)

Summoner ID: kneesyd#NA1
Summoner PUUID: mO4gr-whHun9IRseQslUEGJlORnH1SO3x-uUcAWVjsqyI8GKIQ5oysRDMqRHXSmbG6uzE3m4ZzoO8Q
0.43


# Placing Data Into CSV File

# Loading Data